In [ ]:
#Import packages
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import STOPWORDS


In [ ]:
from tqdm import tqdm
import numpy as np
import gensim # thư viện NLP
import os 
from nltk.tokenize import word_tokenize
doc = []

In [ ]:
dir_path = os.path.dirname(os.path.realpath(os.getcwd()))
data_path = os.path.join(dir_path, 'data\\train_set_ielts')


In [ ]:
data_path


In [ ]:
def rm_stopword(tokenized_doc):
    tok_without_sw = [word for word in tokenized_doc if word.lower() not in STOPWORDS]
    return tok_without_sw

In [ ]:
def get_data(folder_path):
    dirs = os.listdir(folder_path)
    processed_doc = []
    for path in tqdm(dirs):
        file_paths = os.listdir(os.path.join(folder_path, path))
        for file_path in tqdm(file_paths):
            with open(os.path.join(folder_path, path, file_path), 'r',encoding='utf-8') as f:
                lines = f.readlines()
                lines = ' '.join(lines)
                lines = gensim.utils.simple_preprocess(lines)
                lines = ' '.join(lines)

                # Tokenization of each document
                tokenized_doc = word_tokenize(lines.lower())
                #remove stop word
                tokenized_doc = rm_stopword(tokenized_doc, STOPWORDS)
                processed_doc.append([tokenized_doc,path])
    return processed_doc

def run_test(model, test_doc):
    test_pass = 0
    for index in range(len(test_doc)):
        result = model.docvecs.most_similar(positive=[model.infer_vector(test_doc[index][0])],topn=6)
        if (result[0][0] == test_doc[index][1]):
            test_pass+= 1
    return [test_pass,len(test_doc)+1]

In [ ]:
data_train = get_data(data_path)
# data_test = get_data(test_path)

In [ ]:
len(data_train)

In [ ]:
tagged_doc =[]
for case in range(len(data_train)):
    case_i = TaggedDocument(data_train[case][0],[data_train[case][1]])
    tagged_doc.append(case_i)

In [ ]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [ ]:
# Train doc2vec model
model = Doc2Vec(tagged_doc, vector_size=300, window=5, min_count=20, workers=cores, epochs = 50)

In [ ]:
# Save trained doc2vec model
model_path = os.path.dirname(os.path.realpath(os.getcwd()))
model_path = os.path.join(model_path, 'models\\doc2vec.model')
model.save(model_path)

In [ ]:
# Load saved doc2vec model
model= Doc2Vec.load(model_path)

In [ ]:
# find most similar doc topic

text_check = """
India's rice export rates dropped to their lowest in three months on Thursday hit by subdued demand from buyers in Africa, while Vietnamese traders expected falling domestic supplies to cap a further decline in prices.
"""

In [ ]:
model = Doc2Vec.load(model_path)
test_doc = word_tokenize(text_check.lower())
result = model.docvecs.most_similar(positive=[model.infer_vector(test_doc)],topn=6)
result

In [ ]:
from inference import load_model, infer
from gensim.models.doc2vec import Doc2Vec
model = Doc2Vec.load(r"C:\Users\ASUS\Documents\Projects\Python\topic-classification\models\d2v.model")
